GLIDE
====

**GLIDE: Towards Photorealistic Image Generation and Editing with Text-Guided Diffusion Models**

 * Paper: https://arxiv.org/abs/2112.10741
 * Code: https://github.com/openai/glide-text2im

![GLIDE Examples](../assets/glide_examples.jpg)


```bash
conda create -n glide python=3.11 -y
conda activate glide
pip install git+https://github.com/openai/glide-text2im
pip install pyyaml
```

## text-to-image

In [ ]:
from PIL import Image
import torch

from glide_text2im.download import load_checkpoint
from glide_text2im.model_creation import (
    create_model_and_diffusion,
    model_and_diffusion_defaults,
    model_and_diffusion_defaults_upsampler
)

/home/pyml/anaconda3/envs/glide/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
has_cuda = torch.cuda.is_available()
base_steps = "100"  # number of diffusion steps to use for sampling
upsample_steps = "fast27"  # number of diffusion steps to use for upsampling
device = torch.device("cuda" if has_cuda else "cpu")

# Create base model.
options = model_and_diffusion_defaults()
options['use_fp16'] = has_cuda
options['timestep_respacing'] = str(base_steps)

model, diffusion = create_model_and_diffusion(**options)
model.eval()
if has_cuda:
    model.convert_to_fp16()
model.to(device)
model.load_state_dict(load_checkpoint('base', device))
print(' * total base parameters', sum(x.numel() for x in model.parameters()))



# Create upsampler model.
options_up = model_and_diffusion_defaults_upsampler()
options_up['use_fp16'] = has_cuda
options_up['timestep_respacing'] = upsample_steps
model_up, diffusion_up = create_model_and_diffusion(**options_up)
model_up.eval()
if has_cuda:
    model_up.convert_to_fp16()
model_up.to(device)
model_up.load_state_dict(load_checkpoint('upsample', device))
print(' * total upsampler parameters', sum(x.numel() for x in model_up.parameters()))


100%|██████████| 1.54G/1.54G [01:56<00:00, 13.2MiB/s]


 * total base parameters 385030726


100%|██████████| 1.59G/1.59G [02:01<00:00, 13.1MiB/s]


 * total upsampler parameters 398361286
